# data status
- data path: /data/NIA50/50-2/data/NIA50/학습용데이터_pre4/Suwon
- ImageSets
- jsons
- points
- labels

- Classes: {"Car", "Small_Car", "Light_Car", "SUV", "Van", "Small_Truck", "Medium_Truck", "Large_Truck", "Bus", "Mini_Bus", "Special_Vehicle","Two_Wheeler", "Kickboard", "Adult", "Kid"}

In [1]:
# import open3d as o3d
import numpy as np
import cv2
import torch
import pandas as pd
import re
import pickle as pkl
import json
import glob
import shutil
import os
import json
import pathlib as pl
from tqdm import trange, tqdm
from glob import glob
from collections import Counter
from sklearn.model_selection import train_test_split
from scipy.spatial.transform import Rotation as R

CLASSES = ["Car", "Small_Car", "Light_Car", "SUV", "Van", "Small_Truck", "Medium_Truck", "Large_Truck", "Bus", "Mini_Bus", "Special_Vehicle","Two_Wheeler", "Kickboard", "Adult", "Kid"]

def pcd2npy(pcd_path, npy_path, xy2yx: bool = False):
    pcd_f = o3d.t.io.read_point_cloud(pcd_path)
    positions = pcd_f.point.positions.numpy()
    if xy2yx: # x, y, z -> -y, x, z
        positions = positions[:, [1, 0, 2]]
        positions[:, 0] = -positions[:, 0]
    elif xy2yx == False: # x, y, z
        positions = positions[:, [0, 1, 2]]

    intensity = pcd_f.point.intensity.numpy()
    pcd = np.concatenate((positions, intensity), axis = 1)
    np.save(npy_path, pcd)
    
def mkdir(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
data_path = "/data/NIA50/50-2/data/NIA50/nia50_all/raw"
label_list_3d = pl.Path(data_path).glob("*/label/*.json")

In [2]:
label_list_3d = sorted([str(label) for label in label_list_3d])
print((label_list_3d[:5]))

['/data/NIA50/50-2/data/NIA50/nia50_all/raw/L_2210_Suwon_A_A_C0001/label/0000.json', '/data/NIA50/50-2/data/NIA50/nia50_all/raw/L_2210_Suwon_A_A_C0001/label/0001.json', '/data/NIA50/50-2/data/NIA50/nia50_all/raw/L_2210_Suwon_A_A_C0001/label/0002.json', '/data/NIA50/50-2/data/NIA50/nia50_all/raw/L_2210_Suwon_A_A_C0001/label/0003.json', '/data/NIA50/50-2/data/NIA50/nia50_all/raw/L_2210_Suwon_A_A_C0001/label/0004.json']


In [3]:
def get_json_data(json_file_path):
    dp = pd.DataFrame(columns=["class", "position x", "position y", "position z", "rotation", "scale l", "scale w", "scale h", "file_name"])
    for file in label_list_3d:
        with open(file, 'r') as f:
            json_data = json.load(f)
            for i in range(len(json_data)):
                row = [json_data[i]["obj_type"], json_data[i]["psr"]["position"]["x"], json_data[i]["psr"]["position"]["y"], json_data[i]["psr"]["position"]["z"], json_data[i]["psr"]["rotation"]['z'], json_data[i]["psr"]["scale"]["x"], json_data[i]["psr"]["scale"]["y"], json_data[i]["psr"]["scale"]["z"], file.split("/")[-1].split(".")[0]]
                frame_data = pd.DataFrame([row], columns=["class", "position x", "position y", "position z", "rotation", "scale l", "scale w", "scale h", "file_name"])
            dp = pd.concat([dp, frame_data], axis=0)

    # make the dp's index to descending order
    dp = dp.reset_index(drop=True)
    return dp
dp = get_json_data(label_list_3d)

In [ ]:
dp.head(5)

,class,position x,position y,position z,rotation,scale l,scale w,scale h,file_name
0,Car,-17.355507,-36.689805,-10.645174,-0.234196,4.548734,1.654719,1.487743,0000
1,Car,-8.518801,-40.210445,-11.052931,-0.255028,2.061348,0.392235,0.752132,0001
2,Bus,-29.224326,-33.7539,-8.852093,-0.938424,10.788748,2.479926,3.375624,0002
3,Bus,-22.829048,-42.785682,-9.452041,-0.956274,10.735777,2.451729,3.166890,0003
4,SUV,-28.442108,-26.932586,-9.924473,-0.929091,2.956965,0.176704,0.980386,0004


In [ ]:
# Copy dp
dp_edited = dp.copy()

## Show errors

In [ ]:
# show rows if scale l, w, h is 0 or below
dp_edited[(dp_edited["scale l"] <= 0) | (dp_edited["scale w"] <= 0) | (dp_edited["scale h"] <= 0)]

,class,position x,position y,position z,rotation,scale l,scale w,scale h,file_name
294,Adult,41.788159,5.773402,-12.123341,-2.801244,-0.453406,0.23949,0.851781,Suwon_A_2210261635_0129_0004
11350,0,34.074837,2.665231,-12.481919,-1.574237,0.000000,0.00000,0.000000,Suwon_B_2211092244_0058_0000
11874,0,22.075823,-36.560772,-9.516077,-0.875797,0.000000,0.00000,0.000000,Suwon_B_2211121422_0012_0004


In [ ]:
# show if dp's class name is 0
dp_edited[dp_edited["class"] == 0]

,class,position x,position y,position z,rotation,scale l,scale w,scale h,file_name
11350,0,34.074837,2.665231,-12.481919,-1.574237,0.0,0.0,0.0,Suwon_B_2211092244_0058_0000
11874,0,22.075823,-36.560772,-9.516077,-0.875797,0.0,0.0,0.0,Suwon_B_2211121422_0012_0004


In [ ]:
# show the rows if scale l, w is over 10 show the row, if the class is not "Bus" or "Mini_Bus" or "Large_Truck" or "Medium_Truck" or "Small_Truck"
dp_edited[((dp_edited["scale l"] > 10) | (dp_edited["scale w"] > 10) | (dp_edited["scale h"] > 10)) & (dp_edited["class"] != "Bus") & (dp_edited["class"] != "Mini_Bus") & (dp_edited["class"] != "Large_Truck") & (dp_edited["class"] != "Medium_Truck") & (dp_edited["class"] != "Small_Truck")]

,class,position x,position y,position z,rotation,scale l,scale w,scale h,file_name
4502,Van,16.356420,-27.823538,-11.404937,3.115413,11.341120,2.483188,3.446925,Suwon_B_2210261635_0135_0002
4503,Small_Car,-399.441682,179.679106,-12.347579,-2.376728,14.111455,74.973170,1.477866,Suwon_B_2210261635_0135_0003
4504,Small_Car,273.248300,-14.334896,-12.347579,-0.123963,107.548125,70.057268,1.477866,Suwon_B_2210261635_0135_0004
4505,Small_Car,273.248300,-14.334896,-12.347579,-0.123963,107.548125,70.057268,1.477866,Suwon_B_2210261635_0135_0004


In [ ]:
# show if car's scale l, w, h is under 0.5
dp_edited[(dp_edited["class"] == "Car") & ((dp_edited["scale l"] < 0.5) | (dp_edited["scale w"] < 0.5) | (dp_edited["scale h"] < 0.5))]

,class,position x,position y,position z,rotation,scale l,scale w,scale h,file_name
1,Car,40.210445,-8.518801,-11.052931,-0.255028,2.061348,0.392235,0.752132,Suwon_A_2210261635_0000_0001
38,Car,29.764676,-31.219540,-9.694860,-0.910160,3.592732,0.374579,1.205749,Suwon_A_2210261635_0069_0008
83,Car,57.553521,-7.078260,-11.393911,-0.976611,4.031762,1.695949,0.416194,Suwon_A_2210261635_0105_0003
264,Car,35.123443,-22.283066,-10.367157,-0.888539,3.755472,0.321312,1.232393,Suwon_A_2210261635_0126_0004
265,Car,37.454737,25.703558,-13.601653,0.659529,1.335336,0.148141,0.306845,Suwon_A_2210261635_0126_0005
...,...,...,...,...,...,...,...,...,...
12512,Car,23.303860,-31.170620,-9.533151,-0.894679,4.327472,0.360481,1.131262,Suwon_B_2211121422_0165_0002
12516,Car,14.395093,-38.273038,-8.965562,-0.864641,4.305961,0.254443,0.635518,Suwon_B_2211121422_0165_0006
12517,Car,20.592203,-33.241382,-9.440475,-0.890821,3.113877,0.168962,0.664157,Suwon_B_2211121422_0165_0007
12518,Car,27.605414,-27.577062,-9.810832,-0.820463,3.800851,0.425404,0.863612,Suwon_B_2211121422_0165_0008


## fix errors and save it to edited_dp

In [ ]:
# remove the rows if class is not in the CLASSES
dp_edited = dp_edited[dp_edited["class"].isin(CLASSES)]

# remove the rows if scale l, w is over 20
dp_edited = dp_edited[(dp_edited["scale l"] <= 20) & (dp_edited["scale w"] <= 20) & (dp_edited["scale h"] <= 20)]
dp_edited = dp_edited.reset_index(drop=True)

# if the value is under 0, change it to positive value
# if the value is 0, remove it
dp_edited["scale l"] = dp_edited["scale l"].apply(lambda x: abs(x) if x != 0 else np.nan)
dp_edited["scale w"] = dp_edited["scale w"].apply(lambda x: abs(x) if x != 0 else np.nan)
dp_edited["scale h"] = dp_edited["scale h"].apply(lambda x: abs(x) if x != 0 else np.nan)
dp_edited = dp_edited.dropna(axis=0)
dp_edited = dp_edited.reset_index(drop=True)
with open("dp_edited.pkl", "wb") as f:
    pkl.dump(dp_edited, f)

## statistics of data

In [10]:
from collections import Counter

#클래스 종류
classes = Counter(dp_edited["class"])
print(classes)
# position x, y, z 범위
# min, max of position x, y, z
print(f"min of position x: {min(dp_edited['position x']):.2f}, max of position x: {max(dp_edited['position x']):.2f}")
print(f"min of position y: {min(dp_edited['position y']):.2f}, max of position y: {max(dp_edited['position y']):.2f}")
print(f"min of position z: {min(dp_edited['position z']):.2f}, max of position z: {max(dp_edited['position z']):.2f}")

# rotation 범위
print(f"min of rotation: {min(dp_edited['rotation']):.2f}, max of rotation: {max(dp_edited['rotation']):.2f}")

# scale l, w, h 범위
print(f"min of scale l: {min(dp_edited['scale l']):.2f}, max of scale l: {max(dp_edited['scale l']):.2f}")
print(f"min of scale w: {min(dp_edited['scale w']):.2f}, max of scale w: {max(dp_edited['scale w']):.2f}")
print(f"min of scale h: {min(dp_edited['scale h']):.2f}, max of scale h: {max(dp_edited['scale h']):.2f}")

# average of scale l, w, h for each class. Make this to pandas dataframe
# the row's index is class name, and the column's name is average of scale l, w, h
class_scal_avg = pd.DataFrame(columns=["scale l", "scale w", "scale h"])
for c in classes.keys():
    class_scal_avg.loc[c] = [dp_edited[dp_edited["class"] == c]["scale l"].mean(), dp_edited[dp_edited["class"] == c]["scale w"].mean(), dp_edited[dp_edited["class"] == c]["scale h"].mean()]
class_scal_avg

Counter({'Car': 5602, 'Adult': 2469, 'SUV': 1646, 'Two_Wheeler': 675, 'Small_Car': 521, 'Bus': 350, 'Van': 296, 'Light_Car': 280, 'Small_Truck': 256, 'Medium_Truck': 234, 'Kid': 128, 'Special_Vehicle': 61, 'Large_Truck': 36, 'Mini_Bus': 18, 'Kickboard': 3})
min of position x: -49.82, max of position x: 90.37
min of position y: -82.92, max of position y: 53.65
min of position z: -15.62, max of position z: -3.38
min of rotation: -3.14, max of rotation: 3.14
min of scale l: 0.00, max of scale l: 12.91
min of scale w: 0.00, max of scale w: 5.22
min of scale h: 0.01, max of scale h: 3.95


,scale l,scale w,scale h
Car,3.446463,1.208627,1.127142
Bus,9.880856,2.354206,2.937542
SUV,3.794296,1.323224,1.334168
Light_Car,3.153659,1.349812,1.317873
Adult,0.404952,0.504832,1.178620
Small_Truck,4.232092,1.526602,1.783969
Small_Car,3.255399,1.300888,1.283663
Two_Wheeler,1.566077,0.543596,1.181596
Van,4.280662,1.479172,1.572987
Medium_Truck,5.196606,1.786977,2.163979


In [21]:
# min & max of position x, y, z and scale l, w, h for each class. Make this to pandas dataframe
# the row's index is class name, and the column's name is min & max of position x, y, z and scale l, w, h
# each values is .3f
class_min_max = pd.DataFrame(columns=["position x min", "position x max", "position y min", "position y max", "position z min", "position z max", "scale l min", "scale l max", "scale w min", "scale w max", "scale h min", "scale h max", "scale l avg", "scale w avg", "scale h avg"])
for c in classes.keys():
    class_min_max.loc[c] = [f"{min(dp_edited[dp_edited['class'] == c]['position x']):.3f}", 
                            f"{max(dp_edited[dp_edited['class'] == c]['position x']):.3f}", 
                            f"{min(dp_edited[dp_edited['class'] == c]['position y']):.3f}", 
                            f"{max(dp_edited[dp_edited['class'] == c]['position y']):.3f}", 
                            f"{min(dp_edited[dp_edited['class'] == c]['position z']):.3f}", 
                            f"{max(dp_edited[dp_edited['class'] == c]['position z']):.3f}", 
                            f"{min(dp_edited[dp_edited['class'] == c]['scale l']):.3f}", 
                            f"{max(dp_edited[dp_edited['class'] == c]['scale l']):.3f}", 
                            f"{min(dp_edited[dp_edited['class'] == c]['scale w']):.3f}", 
                            f"{max(dp_edited[dp_edited['class'] == c]['scale w']):.3f}", 
                            f"{min(dp_edited[dp_edited['class'] == c]['scale h']):.3f}", 
                            f"{max(dp_edited[dp_edited['class'] == c]['scale h']):.3f}", 
                            f"{class_scal_avg.loc[c]['scale l']:.3f}", 
                            f"{class_scal_avg.loc[c]['scale w']:.3f}", 
                            f"{class_scal_avg.loc[c]['scale h']:.3f}"]
class_min_max

,position x min,position x max,position y min,position y max,position z min,position z max,scale l min,scale l max,scale w min,scale w max,scale h min,scale h max,scale l avg,scale w avg,scale h avg
Car,-49.815,74.968,-82.923,44.715,-15.622,-3.378,0.004,6.624,0.004,5.220,0.005,3.245,3.446,1.209,1.127
Bus,3.941,90.372,-45.283,47.777,-13.966,-7.439,0.100,12.914,0.105,4.421,0.183,3.894,9.881,2.354,2.938
SUV,-3.224,61.616,-61.270,44.188,-15.162,-8.326,0.145,5.798,0.027,2.494,0.011,2.496,3.794,1.323,1.334
Light_Car,-2.391,56.186,-42.034,38.011,-14.765,-8.430,0.352,8.038,0.101,3.398,0.064,1.988,3.154,1.350,1.318
Adult,4.746,54.790,-43.396,20.283,-13.148,-8.662,0.015,2.929,0.024,4.077,0.006,2.637,0.405,0.505,1.179
Small_Truck,-14.197,58.038,-39.388,42.710,-14.957,-8.285,0.419,7.122,0.071,3.212,0.101,3.435,4.232,1.527,1.784
Small_Car,-1.054,53.613,-40.993,40.163,-14.834,-8.851,0.308,6.192,0.037,2.276,0.033,1.961,3.255,1.301,1.284
Two_Wheeler,5.903,54.171,-45.769,38.749,-15.064,-8.732,0.181,2.434,0.078,1.560,0.046,1.963,1.566,0.544,1.182
Van,-1.186,47.195,-40.137,53.651,-14.453,-8.542,0.165,11.341,0.016,3.047,0.015,3.447,4.281,1.479,1.573
Medium_Truck,7.692,55.170,-42.052,50.391,-14.572,-7.955,0.348,8.747,0.040,3.029,0.014,3.575,5.197,1.787,2.164


In [15]:
# save the edited data
dp_edited.to_csv("data_info.csv")

# Data status 2

In [26]:
from pathlib import Path
from glob import glob

data_path2 = "/data/NIA50/NIA50/50-2/data/nia50_all/raw"
move_to_path = "/data/NIA50/50-2/data/NIA50/학습용데이터_pre5"

# get the path of whole json files including subfolders of it and exclude when the path contains "calib"
json_paths = [str(p) for p in Path(data_path2).rglob("*.json") if "calib" not in str(p) and "2d_label" not in str(p)]
print(json_paths[:5], json_paths[-5:])

['/data/NIA50/NIA50/50-2/data/nia50_all/raw/L_2211_Suwon_B_F_C0274/label/0008.json', '/data/NIA50/NIA50/50-2/data/nia50_all/raw/L_2211_Suwon_B_F_C0274/label/0007.json', '/data/NIA50/NIA50/50-2/data/nia50_all/raw/L_2211_Suwon_B_F_C0274/label/0001.json', '/data/NIA50/NIA50/50-2/data/nia50_all/raw/L_2211_Suwon_B_F_C0274/label/0000.json', '/data/NIA50/NIA50/50-2/data/nia50_all/raw/L_2211_Suwon_B_F_C0274/label/0009.json'] ['/data/NIA50/NIA50/50-2/data/nia50_all/raw/L_2210_Suwon_A_A_C0172/label/0003.json', '/data/NIA50/NIA50/50-2/data/nia50_all/raw/L_2210_Suwon_A_A_C0172/label/0002.json', '/data/NIA50/NIA50/50-2/data/nia50_all/raw/L_2210_Suwon_A_A_C0172/label/0005.json', '/data/NIA50/NIA50/50-2/data/nia50_all/raw/L_2210_Suwon_A_A_C0172/label/0004.json', '/data/NIA50/NIA50/50-2/data/nia50_all/raw/L_2210_Suwon_A_A_C0172/label/0006.json']


In [ ]:
# pcd file path
pcd_paths = [p.replace("json", "pcd") for p in json_paths]

In [ ]:
# copy